In [1]:
# pip install langchain_community

In [2]:
# pip install langchain

In [3]:
# pip install langchain_groq

In [4]:
# pip install sentence_transformers

In [5]:
# pip install faiss-cpu

In [1]:
import json
import os
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_groq import ChatGroq

# Load JSONL Data
data_file = "Bigdata.jsonl"
documents = []

with open(data_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        try:
            doc = json.loads(line.strip())  # Parse JSON
            
            if "prompt" in doc and "completion" in doc:
                content = f"Q: {doc['prompt']}\nA: {doc['completion']}"
                documents.append(Document(page_content=content, metadata={}))

        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line {i}: {e}")

print(f"Total Documents Loaded: {len(documents)}")

# Initialize Embeddings Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Text Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
split_docs = text_splitter.split_documents(documents)

# Create FAISS Vector Store
vectorstore = FAISS.from_documents(split_docs, embedding=embedding_model)

# Initialize Groq LLM
groq_api_key = os.getenv("GROQ_API_KEY")

Total Documents Loaded: 5000


C:\Users\arpit\AppData\Local\Temp\ipykernel_9244\3351171350.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\arpit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatGroq(model_name="deepseek-r1-distill-llama-70b", api_key=groq_api_key)

# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
)

# Chatbot Function
def chatbot(query):
    return qa_chain.run(query)

# Example Usage
if __name__ == "__main__":
    user_query = "Any advice on coconut fruit drop?"
    print(chatbot(user_query))


C:\Users\arpit\AppData\Local\Temp\ipykernel_1380\3147163430.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa_chain.run(query)


<think>
Okay, so I need to figure out how to answer the question about coconut fruit drop. Let me start by looking at the context provided.

First, I see that the problem of immature fruit drop in coconuts can be effectively controlled. That's good news. The context mentions that by taking certain steps, one can reduce fruit drop and improve yield and quality. 

Looking at the Q&A sections, the first Q&A says that coconut fruit drop is caused by various reasons like nutritional deficiencies, pests, diseases, and unsuitable climate. The advice given is to apply boron fertilizer—50 grams per tree to improve fruit quality and reduce falling.

The second Q&A is about premature fruit drop. It suggests applying 25 grams of borax along with cow dung around the trunk of each tree. This is said to help reduce fruit falling.

The third Q&A is similar to the first two, asking about premature fruit drop and getting the same advice as the second.

So, the user is asking, "Any advice on coconut frui

In [ ]:
# new
# from langchain.evaluation import load_evaluator

# evaluator = load_evaluator("retrieval", k=5)
# retrieval_score = evaluator.evaluate(
#     predictions=[d.page_content for d in documents], 
#     references=[d.page_content for d in split_docs]
# )
# print("Retrieval Score:", retrieval_score)


ValueError: Unknown evaluator type: retrieval
Valid types are: [<EvaluatorType.QA: 'qa'>, <EvaluatorType.COT_QA: 'cot_qa'>, <EvaluatorType.CONTEXT_QA: 'context_qa'>, <EvaluatorType.PAIRWISE_STRING: 'pairwise_string'>, <EvaluatorType.SCORE_STRING: 'score_string'>, <EvaluatorType.LABELED_PAIRWISE_STRING: 'labeled_pairwise_string'>, <EvaluatorType.LABELED_SCORE_STRING: 'labeled_score_string'>, <EvaluatorType.AGENT_TRAJECTORY: 'trajectory'>, <EvaluatorType.CRITERIA: 'criteria'>, <EvaluatorType.LABELED_CRITERIA: 'labeled_criteria'>, <EvaluatorType.STRING_DISTANCE: 'string_distance'>, <EvaluatorType.PAIRWISE_STRING_DISTANCE: 'pairwise_string_distance'>, <EvaluatorType.EMBEDDING_DISTANCE: 'embedding_distance'>, <EvaluatorType.PAIRWISE_EMBEDDING_DISTANCE: 'pairwise_embedding_distance'>, <EvaluatorType.JSON_VALIDITY: 'json_validity'>, <EvaluatorType.JSON_EQUALITY: 'json_equality'>, <EvaluatorType.JSON_EDIT_DISTANCE: 'json_edit_distance'>, <EvaluatorType.JSON_SCHEMA_VALIDATION: 'json_schema_vali

In [8]:
llm = ChatGroq(model_name="deepseek-r1-distill-llama-70b", api_key=groq_api_key)

# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
)

# Chatbot Function
def chatbot(query):
    return qa_chain.run(query)

# Example Usage
if __name__ == "__main__":
    user_query = "Any advice on coconut fruit drop?"
    print(chatbot(user_query))


<think>
Okay, so I need to figure out how to help someone who's having issues with their coconut trees dropping fruit prematurely. I remember reading that there are a few reasons why this might happen, like not enough nutrients, pests, diseases, or maybe the weather isn't right. 

First, I think about what the user is asking. They want advice on stopping the fruit from falling too early. I recall that boron deficiency is a common issue in coconut trees. So maybe suggesting a boron fertilizer would help. I think the amount was around 50 grams per tree. That makes sense because boron is important for plant health and can prevent such problems.

Another thing I remember is using borax with cow dung. I think the quantity was 25 grams of borax mixed with some cow dung. Applying that around the base of the tree might provide the necessary nutrients and improve the soil condition, which in turn could reduce the fruit drop.

I should also consider other factors like making sure the trees have 

In [13]:
llm = ChatGroq(model_name="qwen-2.5-32b", api_key=groq_api_key)

# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
)

# Chatbot Function
def chatbot(query):
    return qa_chain.run(query)

# Example Usage
if __name__ == "__main__":
    user_query = "How to control rice borer"
    print(chatbot(user_query))


To control rice borers, consider implementing the following strategies:

1. **Chemical Control**: Use specific pesticides like Classic 20. As an example, mix 2 ml of Classic 20 with a liter of water and spray it on the infested areas. This pesticide should help in killing the rice borers.

2. **Hygiene and Cleanliness**: Keep the growing environment clean by removing weeds and crop residues that can serve as breeding grounds for borers.

3. **Proper Irrigation and Fertilization**: Maintain the health of the rice plants by ensuring they receive adequate water and nutrients. Healthy plants are more resistant to pest attacks.

4. **Crop Management Practices**: Implement crop rotation and intercropping to reduce borer populations and minimize the chances of infestation.

By following these methods, you can effectively manage and reduce the impact of rice borers on your rice crop.


In [10]:
llm = ChatGroq(model_name="gemma2-9b-it", api_key=groq_api_key)

# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
)

# Chatbot Function
def chatbot(query):
    return qa_chain.run(query)

# Example Usage
if __name__ == "__main__":
    user_query = "Any advice on coconut fruit drop?"
    print(chatbot(user_query))


Here is some advice on coconut fruit drop: 

Coconut fruit drop may be caused by a variety of reasons, including nutritional deficiencies, pests and diseases, unsuitable climatic conditions, etc. To reduce fruit drop, it is recommended that you take the following measures:
1. Apply boron fertilizer: Applying 50 grams of boron fertilizer to each coconut tree can help improve fruit quality and reduce falling. 

If your coconut palm fruit is falling prematurely, you may consider the following measures:
Apply 25 grams of borax and cow dung to each coconut tree: Sprinkle 25 grams of borax and an appropriate amount of cow dung around the trunk of each coconut tree, which will help reduce fruit falling. 




In [11]:
llm = ChatGroq(model_name="mixtral-8x7b-32768", api_key=groq_api_key)

# RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
)

# Chatbot Function
def chatbot(query):
    return qa_chain.run(query)

# Example Usage
if __name__ == "__main__":
    user_query = "Any advice on coconut fruit drop?"
    print(chatbot(user_query))


If fruit is falling prematurely from coconut trees, you can try applying boron fertilizer and cow dung to each tree. Specifically, you can apply 50 grams of boron fertilizer and an appropriate amount of cow dung to each tree. Additionally, sprinkling 25 grams of borax around the trunk of each tree may also help reduce fruit falling. These measures can help improve fruit quality and reduce premature fruit drop, resulting in increased yield and quality of coconuts.
